<a href="https://colab.research.google.com/github/Vikash-Chaubey7061/DATA-SCIENCE-PROJECT-USING-PYTHON/blob/main/Food_waste_management.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Project Title- Project Title	Local Food Wastage Management System

Project By- Vikash Kumar Chaubey

Skills take away From This Project                
●	Python
●	SQL
●	Streamlit
●	Data Analysis


Domain

●	Food Management
●	Waste Reduction
●	Social Good.


Problem Statement-    Food wastage is a significant issue, with many households and restaurants discarding surplus food while numerous people struggle with food insecurity. This project aims to develop a Local Food Wastage Management System, where:
●	Restaurants and individuals can list surplus food.
●	NGOs or individuals in need can claim the food.
●	SQL stores available food details and locations.
●	A Streamlit app enables interaction, filtering, CRUD operation and visualization.


Database Setup (SQL)

In [1]:
import sqlite3
import pandas as pd

# Create SQLite database
conn = sqlite3.connect('food_waste.db')
cursor = conn.cursor()

# Create Providers table
cursor.execute('''
CREATE TABLE IF NOT EXISTS Providers (
    Provider_ID INTEGER PRIMARY KEY,
    Name TEXT,
    Type TEXT,
    Address TEXT,
    City TEXT,
    Contact TEXT
)
''')

# Create Receivers table
cursor.execute('''
CREATE TABLE IF NOT EXISTS Receivers (
    Receiver_ID INTEGER PRIMARY KEY,
    Name TEXT,
    Type TEXT,
    City TEXT,
    Contact TEXT
)
''')

# Create Food_Listings table
cursor.execute('''
CREATE TABLE IF NOT EXISTS Food_Listings (
    Food_ID INTEGER PRIMARY KEY,
    Food_Name TEXT,
    Quantity INTEGER,
    Expiry_Date DATE,
    Provider_ID INTEGER,
    Provider_Type TEXT,
    Location TEXT,
    Food_Type TEXT,
    Meal_Type TEXT,
    FOREIGN KEY (Provider_ID) REFERENCES Providers(Provider_ID)
)
''')

# Create Claims table
cursor.execute('''
CREATE TABLE IF NOT EXISTS Claims (
    Claim_ID INTEGER PRIMARY KEY,
    Food_ID INTEGER,
    Receiver_ID INTEGER,
    Status TEXT,
    Timestamp DATETIME,
    FOREIGN KEY (Food_ID) REFERENCES Food_Listings(Food_ID),
    FOREIGN KEY (Receiver_ID) REFERENCES Receivers(Receiver_ID)
)
''')

# Load data from CSV files (assuming you've downloaded them)
def load_csv_to_table(csv_file, table_name):
    df = pd.read_csv(csv_file)
    df.to_sql(table_name, conn, if_exists='append', index=False)

# Example usage (replace with actual file paths):
# load_csv_to_table('providers_data.csv', 'Providers')
# load_csv_to_table('receivers_data.csv', 'Receivers')
# load_csv_to_table('food_listings_data.csv', 'Food_Listings')
# load_csv_to_table('claims_data.csv', 'Claims')

conn.commit()

2. SQL Queries Implementation

In [2]:
def execute_query(query, params=None):
    cursor = conn.cursor()
    if params:
        cursor.execute(query, params)
    else:
        cursor.execute(query)
    return cursor.fetchall()

# Query 1: Food providers and receivers in each city
query1 = '''
SELECT
    p.City,
    COUNT(DISTINCT p.Provider_ID) AS Provider_Count,
    COUNT(DISTINCT r.Receiver_ID) AS Receiver_Count
FROM Providers p
LEFT JOIN Receivers r ON p.City = r.City
GROUP BY p.City
'''

# Query 2: Provider type contributing most food
query2 = '''
SELECT
    p.Type,
    SUM(fl.Quantity) AS Total_Quantity
FROM Providers p
JOIN Food_Listings fl ON p.Provider_ID = fl.Provider_ID
GROUP BY p.Type
ORDER BY Total_Quantity DESC
LIMIT 1
'''

# Query 5: Total quantity of food available
query5 = '''
SELECT SUM(Quantity) AS Total_Food_Available
FROM Food_Listings
'''

# Query 8: Food claims per food item
query8 = '''
SELECT
    fl.Food_Name,
    COUNT(c.Claim_ID) AS Claim_Count
FROM Food_Listings fl
LEFT JOIN Claims c ON fl.Food_ID = c.Food_ID
GROUP BY fl.Food_ID, fl.Food_Name
ORDER BY Claim_Count DESC
'''

# Execute and print results
print("Providers and Receivers per City:")
print(execute_query(query1))
print("\nProvider Type Contributing Most Food:")
print(execute_query(query2))
print("\nTotal Food Available:")
print(execute_query(query5))
print("\nFood Claims per Item:")
print(execute_query(query8))

Providers and Receivers per City:
[]

Provider Type Contributing Most Food:
[]

Total Food Available:
[(None,)]

Food Claims per Item:
[]


3. Streamlit Application

In [5]:
import streamlit as st
import pandas as pd
import sqlite3

# Initialize connection
conn = sqlite3.connect('food_waste.db')

# App title
st.title('Local Food Wastage Management System')

# Sidebar filters
st.sidebar.header('Filters')
city_filter = st.sidebar.selectbox('Select City', ['All'] + list(pd.read_sql('SELECT DISTINCT City FROM Providers', conn)['City']))
food_type_filter = st.sidebar.selectbox('Select Food Type', ['All'] + list(pd.read_sql('SELECT DISTINCT Food_Type FROM Food_Listings', conn)['Food_Type']))
meal_type_filter = st.sidebar.selectbox('Select Meal Type', ['All'] + list(pd.read_sql('SELECT DISTINCT Meal_Type FROM Food_Listings', conn)['Meal_Type']))

# Build query based on filters
query = 'SELECT * FROM Food_Listings WHERE 1=1'
params = []

if city_filter != 'All':
    query += ' AND Location = ?'
    params.append(city_filter)
if food_type_filter != 'All':
    query += ' AND Food_Type = ?'
    params.append(food_type_filter)
if meal_type_filter != 'All':
    query += ' AND Meal_Type = ?'
    params.append(meal_type_filter)

# Display filtered food listings
st.header('Available Food Listings')
food_listings = pd.read_sql(query, conn, params=params if params else None)
st.dataframe(food_listings)

# Display providers in selected city
if city_filter != 'All':
    st.header(f'Food Providers in {city_filter}')
    providers = pd.read_sql('SELECT Name, Type, Contact FROM Providers WHERE City = ?', conn, params=[city_filter])
    st.dataframe(providers)

# CRUD Operations
st.sidebar.header('Manage Listings')
operation = st.sidebar.selectbox('Operation', ['Add', 'Update', 'Delete'])

if operation == 'Add':
    st.header('Add New Food Listing')
    with st.form("add_form"):
        food_name = st.text_input("Food Name")
        quantity = st.number_input("Quantity", min_value=1)
        expiry_date = st.date_input("Expiry Date")
        provider_id = st.number_input("Provider ID", min_value=1)
        food_type = st.selectbox("Food Type", ["Vegetarian", "Non-Vegetarian", "Vegan"])
        meal_type = st.selectbox("Meal Type", ["Breakfast", "Lunch", "Dinner", "Snacks"])

        if st.form_submit_button("Add Listing"):
            cursor = conn.cursor()
            cursor.execute('''
            INSERT INTO Food_Listings (Food_Name, Quantity, Expiry_Date, Provider_ID, Food_Type, Meal_Type, Location)
            VALUES (?, ?, ?, ?, ?, ?,
                (SELECT City FROM Providers WHERE Provider_ID = ?))
            ''', (food_name, quantity, expiry_date, provider_id, food_type, meal_type, provider_id))
            conn.commit()
            st.success("Food listing added successfully!")

# Add more CRUD operations and other features as needed

# Data Analysis Section
st.header('Food Wastage Analysis')

# Query 2 visualization
st.subheader('Food Contribution by Provider Type')
provider_contrib = pd.read_sql('''
SELECT p.Type, SUM(fl.Quantity) AS Total_Quantity
FROM Providers p
JOIN Food_Listings fl ON p.Provider_ID = fl.Provider_ID
GROUP BY p.Type
ORDER BY Total_Quantity DESC
''', conn)
st.bar_chart(provider_contrib.set_index('Type'))

# Query 8 visualization
st.subheader('Most Claimed Food Items')
claimed_food = pd.read_sql('''
SELECT fl.Food_Name, COUNT(c.Claim_ID) AS Claim_Count
FROM Food_Listings fl
LEFT JOIN Claims c ON fl.Food_ID = c.Food_ID
GROUP BY fl.Food_ID, fl.Food_Name
ORDER BY Claim_Count DESC
LIMIT 10
''', conn)
st.bar_chart(claimed_food.set_index('Food_Name'))

2025-08-16 15:20:19.607 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-16 15:20:19.884 
  command:

    streamlit run /usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2025-08-16 15:20:19.888 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-16 15:20:19.889 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-16 15:20:19.893 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-16 15:20:19.896 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-16 15:20:19.898 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-16 15:20:19.917 Thread 'MainThread': mi

DeltaGenerator(_form_data=FormData(form_id='add_form'))

In [4]:
!pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 99.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 102.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 6.3 MB/s eta 0:00:00


In [7]:
streamlit run Local_Food_Wastage_Management_System_app.py

SyntaxError: invalid syntax (ipython-input-3011666378.py, line 1)

In [8]:
%%writefile app.py
import streamlit as st
import pandas as pd
import sqlite3

# Initialize connection
conn = sqlite3.connect('food_waste.db')

# App title
st.title('Local Food Wastage Management System')

# Sidebar filters
st.sidebar.header('Filters')
city_filter = st.sidebar.selectbox('Select City', ['All'] + list(pd.read_sql('SELECT DISTINCT City FROM Providers', conn)['City']))
food_type_filter = st.sidebar.selectbox('Select Food Type', ['All'] + list(pd.read_sql('SELECT DISTINCT Food_Type FROM Food_Listings', conn)['Food_Type']))
meal_type_filter = st.sidebar.selectbox('Select Meal Type', ['All'] + list(pd.read_sql('SELECT DISTINCT Meal_Type FROM Food_Listings', conn)['Meal_Type']))

# Build query based on filters
query = 'SELECT * FROM Food_Listings WHERE 1=1'
params = []

if city_filter != 'All':
    query += ' AND Location = ?'
    params.append(city_filter)
if food_type_filter != 'All':
    query += ' AND Food_Type = ?'
    params.append(food_type_filter)
if meal_type_filter != 'All':
    query += ' AND Meal_Type = ?'
    params.append(meal_type_filter)

# Display filtered food listings
st.header('Available Food Listings')
food_listings = pd.read_sql(query, conn, params=params if params else None)
st.dataframe(food_listings)

# Display providers in selected city
if city_filter != 'All':
    st.header(f'Food Providers in {city_filter}')
    providers = pd.read_sql('SELECT Name, Type, Contact FROM Providers WHERE City = ?', conn, params=[city_filter])
    st.dataframe(providers)

# CRUD Operations
st.sidebar.header('Manage Listings')
operation = st.sidebar.selectbox('Operation', ['Add', 'Update', 'Delete'])

if operation == 'Add':
    st.header('Add New Food Listing')
    with st.form("add_form"):
        food_name = st.text_input("Food Name")
        quantity = st.number_input("Quantity", min_value=1)
        expiry_date = st.date_input("Expiry Date")
        provider_id = st.number_input("Provider ID", min_value=1)
        food_type = st.selectbox("Food Type", ["Vegetarian", "Non-Vegetarian", "Vegan"])
        meal_type = st.selectbox("Meal Type", ["Breakfast", "Lunch", "Dinner", "Snacks"])

        if st.form_submit_button("Add Listing"):
            cursor = conn.cursor()
            cursor.execute('''
            INSERT INTO Food_Listings (Food_Name, Quantity, Expiry_Date, Provider_ID, Food_Type, Meal_Type, Location)
            VALUES (?, ?, ?, ?, ?, ?,
                (SELECT City FROM Providers WHERE Provider_ID = ?))
            ''', (food_name, quantity, expiry_date, provider_id, food_type, meal_type, provider_id))
            conn.commit()
            st.success("Food listing added successfully!")

# Add more CRUD operations and other features as needed

# Data Analysis Section
st.header('Food Wastage Analysis')

# Query 2 visualization
st.subheader('Food Contribution by Provider Type')
provider_contrib = pd.read_sql('''
SELECT p.Type, SUM(fl.Quantity) AS Total_Quantity
FROM Providers p
JOIN Food_Listings fl ON p.Provider_ID = fl.Provider_ID
GROUP BY p.Type
ORDER BY Total_Quantity DESC
''', conn)
st.bar_chart(provider_contrib.set_index('Type'))

# Query 8 visualization
st.subheader('Most Claimed Food Items')
claimed_food = pd.read_sql('''
SELECT fl.Food_Name, COUNT(c.Claim_ID) AS Claim_Count
FROM Food_Listings fl
LEFT JOIN Claims c ON fl.Food_ID = c.Food_ID
GROUP BY fl.Food_ID, fl.Food_Name
ORDER BY Claim_Count DESC
LIMIT 10
''', conn)
st.bar_chart(claimed_food.set_index('Food_Name'))

Writing app.py


In [10]:
!streamlit run app.py --browser.gatherUsageStats false


  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.21.120.73:8501

  Stopping...
  Stopping...
